In [ ]:

def parse_survey_answers_files():
    df = pd.read_csv('./data/survey.csv')
    df.dropna()
    df = df.drop(
        ['W skali od 1 do 10 jak bardzo lubisz słone jedzenie', 'W skali od 1 do 10 jak bardzo lubisz słodkie jedzenie',
         'W skali od 1 do 10 jak bardzo lubisz gorzkie jedzenie', 'W skali od 1 do 10 jak bardzo lubisz mięso',
         'W skali od 1 do 10 jak bardzo lubisz pikantne jedzenie',
         'W skali od 1 do 10 jak bardzo lubisz kuchnie śródziemnomorską',
         'W skali od 1 do 10 jak bardzo lubisz kuchnie azjatycką',
         'W skali od 1 do 10 jak bardzo lubisz kuchnie amerykańską',
         'W skali od 1 do 10 jak bardzo lubisz kuchnie arabską',
         'W skali od 1 do 10 jak bardzo lubisz owoce morza(również ryby)',
         'W skali od 1 do 10 jak bardzo lubisz tłuste jedzenie', 'W skali od 1 do 10 jak bardzo lubisz kuchnie Polską'
         ],
        axis=1)
    columns = df.columns
    df.T.insert(0, "questions", columns)
    return df

parse_survey_answers_files().head()


In [1]:
import pandas as pd
from src.utils import load_and_preprocess_data
from engine.recommender import Recommender
from scipy import spatial

In [4]:
data, email_order, dishes_order, sorted_users, sorted_dishes = load_and_preprocess_data(ratings_path = './data/ratings.csv')
recommender = Recommender(
        data['ocena'],
        email_order,
        dishes_order,
    )

In [8]:
recommender.create_and_fit(
        # Fine-tuned values
        model_params=dict(
            factors=40,
            alpha=0.6,
            regularization=0.06,
        ),
)

F:\uczelnia\AI_DIET_GENERATOR\rec_system\venv\lib\site-packages\implicit\utils.py:138: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0 seconds
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

In [9]:
suggestions_and_score  = recommender.recommend_products(
                20, 40
            )


# for i in range(len(suggestions_and_score[1])):
#     print(suggestions_and_score[0][i], "   ", suggestions_and_score[1][i])
#     print(sorted_dishes[suggestions_and_score[0][i]])

In [10]:
def compare_taste_with_taste_profile(dish_name_list, user_email, user_profiles_df: pd.DataFrame = None,
                                     user_profiles_path: str = './src/data/user_profiles.csv',
                                     recipes_df: pd.DataFrame = None, recipes_path: str = './data/recipes.csv'):
    if not user_profiles_df:
        user_profiles_df = pd.read_csv(user_profiles_path)
    user_profile = (user_profiles_df.loc[user_profiles_df['email'] == user_email][
                        ["saltiness", "bitterness", 'spiciness', 'fattiness']].values * 10)[0]
    if not recipes_df:
        recipes_df = pd.read_csv(recipes_path)
    cosine_similarity_list = list()
    for dish_name in dish_name_list:
        dish = \
            recipes_df.loc[recipes_df['title'] == dish_name][
                ["saltiness", "bitterness", 'spiciness', 'fattiness']].values[
                0]
        cosine_similarity_list.append((1 - spatial.distance.cosine(user_profile, dish), dish_name))
    cosine_similarity_list.sort(key=lambda x: x[0], reverse=True)
    return cosine_similarity_list


compare_taste_with_taste_profile([sorted_dishes[i] for i in suggestions_and_score[0].tolist()], sorted_users[20])


[(0.8304249455317739, "Jen's Swedish Meatballs"),
 (0.8276961252801738, 'Beef Wellington'),
 (0.8270715804662274, 'Vegan Colcannon Soup'),
 (0.826416979086038, 'X-Country Double Lobster Risotto'),
 (0.8255045512803283, 'Colcannon'),
 (0.8249217329670919, 'Slow Cooker Shanghai-Style Beef Borscht'),
 (0.8248401403603774, 'Austrian Goulash'),
 (0.8208052386056948, 'Hungarian Beef Goulash'),
 (0.8181758430417793, 'Baked Chicken Caprese'),
 (0.8161270368569296,
  'Spring Risotto with Shrimp, Asparagus and Artichoke Hearts'),
 (0.8138330212896766, 'Gluten-Free Lamb Moussaka'),
 (0.8090427296626866, 'Clams With Spanish Sausage'),
 (0.8056793300738222, 'Sauteed Italian Eggplant Side Dish'),
 (0.7958477017765462, 'Pierogi Casserole'),
 (0.7926557231657605, 'Italian Seafood Stew'),
 (0.7878684138210567, 'Light Greek Lemon Chicken Orzo Soup'),
 (0.7857365736918617, 'Mushroom and Parmesan Brown Rice Risotto'),
 (0.7691188553638618, 'Beef Stroganoff w Bella Mushrooms'),
 (0.7501219704212928, 'Spani

In [13]:
from data.survey import update_survey_answers_file
test = update_survey_answers_file(to_csv=False)

In [26]:
import pandas as pd
from data.survey import convert_to_implicit_dataset
test.to_csv('test1')
df = convert_to_implicit_dataset(survey_path='./test1', to_csv=True, destination='./tesldfgdfgt.csv')
df

,Adres e-mail,pytanie,ocena
0,tast@gmail.com,Unnamed: 0,0
1,sylweczkaa@buziaczek.pl,Unnamed: 0,1
2,ola.zarzecka96@gmail.com,Unnamed: 0,2
3,lukaszzak99@wp.pl,Unnamed: 0,3
4,s20487@pjwstk.edu.pl,Unnamed: 0,0
...,...,...,...
13795,borkowskaj2000@gmail.com,Pepperoni Pizza Muffins,8.0
13796,agakucha7@gmail.com,Pepperoni Pizza Muffins,4.0
13797,aleksandraslesarchuk@gmail.com,Pepperoni Pizza Muffins,6.0
13798,annaskolimowska4@gmail.com,Pepperoni Pizza Muffins,5.0


In [21]:
pd.read_csv('data/survey.csv')

,Adres e-mail,W skali od 1 do 10 jak bardzo lubisz słone jedzenie,W skali od 1 do 10 jak bardzo lubisz słodkie jedzenie,W skali od 1 do 10 jak bardzo lubisz gorzkie jedzenie,W skali od 1 do 10 jak bardzo lubisz mięso,W skali od 1 do 10 jak bardzo lubisz pikantne jedzenie,W skali od 1 do 10 jak bardzo lubisz kuchnie śródziemnomorską,W skali od 1 do 10 jak bardzo lubisz kuchnie azjatycką,W skali od 1 do 10 jak bardzo lubisz kuchnie amerykańską,W skali od 1 do 10 jak bardzo lubisz kuchnie arabską,...,"Red, White & Blue Crepes: Happy July 4th! @driscollsberry",Layered Baked Eggplant Parmesan,Rustic Grilled Peaches Pizza,Pork Schnitzel And Apple Salad,Lightened Ricotta Stuffed Shells With Italian Sausage Ragu,Buffalo Mozzarella & Pepperoni Pizza with a Cauliflower Crust,Coriander Ravioli With Pumpkin & Cottage Cheese Filling,Fig and Goat Cheese Pizza With Pesto,Classic French Onion Soup,Pepperoni Pizza Muffins
0,tast@gmail.com,7,8,3,9,7,9,8,7,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,sylweczkaa@buziaczek.pl,9,8,2,1,3,10,2,7,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ola.zarzecka96@gmail.com,10,8,2,7,7,10,10,7,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,lukaszzak99@wp.pl,9,7,5,10,10,7,7,9,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,s20487@pjwstk.edu.pl,8,6,3,8,10,9,10,6,9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,pawel280498@interia.pl,7,5,2,8,5,4,9,7,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,matysiakmonik@gmail.com,10,10,8,10,10,10,10,7,10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,s20698@pjwstk.edu.pl,10,4,2,8,3,8,10,7,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,alexandrablack369@gmail.com,10,10,5,10,7,7,8,7,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,thebotz9@gmail.com,8,3,5,8,5,8,9,9,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:

df = convert_to_implicit_dataset(survey_path='./data/survey.csv', to_csv=True, destination='./tesldfgdfgt.csv')
df

,Adres e-mail,pytanie,ocena
0,tast@gmail.com,Avocado Chicken Parmigiana,6.0
1,sylweczkaa@buziaczek.pl,Avocado Chicken Parmigiana,1.0
2,ola.zarzecka96@gmail.com,Avocado Chicken Parmigiana,9.0
3,lukaszzak99@wp.pl,Avocado Chicken Parmigiana,3.0
19,wojtek.piorecki@gmail.com,Avocado Chicken Parmigiana,7.0
...,...,...,...
13277,borkowskaj2000@gmail.com,Pepperoni Pizza Muffins,8.0
13278,agakucha7@gmail.com,Pepperoni Pizza Muffins,4.0
13279,aleksandraslesarchuk@gmail.com,Pepperoni Pizza Muffins,6.0
13280,annaskolimowska4@gmail.com,Pepperoni Pizza Muffins,5.0


In [6]:
df

,Adres e-mail,pytanie,ocena
24,0 tast@gmail.com 0 s20...,Sheet Pan Butternut Squash with Spicy Italian ...,5.0
25,1 sylweczkaa@buziaczek.pl 1 paw...,Sheet Pan Butternut Squash with Spicy Italian ...,7.0
26,2 ola.zarzecka96@gmail.com 2 maty...,Sheet Pan Butternut Squash with Spicy Italian ...,3.0
27,3 lukaszzak99@wp.pl 3 ...,Sheet Pan Butternut Squash with Spicy Italian ...,7.0
28,4 lauri.cat@op.pl 4 thea...,Sheet Pan Butternut Squash with Spicy Italian ...,7.0
...,...,...,...
1369,1 sylweczkaa@buziaczek.pl 1 paw...,Pepperoni Pizza Muffins,8.0
1370,2 ola.zarzecka96@gmail.com 2 maty...,Pepperoni Pizza Muffins,4.0
1371,3 lukaszzak99@wp.pl 3 ...,Pepperoni Pizza Muffins,6.0
1372,4 lauri.cat@op.pl 4 thea...,Pepperoni Pizza Muffins,5.0


In [ ]:
from data.recipes import get_all_recipes
test = get_all_recipes('f1a65dd7832647d18888c171f92ed1b7')